# Exercise: Classification of Occupational Descriptions

1. Reproduce the HISCO classification model from the slides by downloading and loading the datasets and running the associated code (see [Exercise 1](##exercise-1))
1. Try changing the `nn.GRU` layer to an `nn.RNN` layer and then to an `nn.LSTM` layer and train your new models
1. Try changing the optimizer to something other than `torch.optim.AdamW`. A list of optimizers is available [here](https://pytorch.org/docs/stable/optim.html#algorithms)
1. Experiment with, e.g., the types or numbers of layers in your model, the choice of optimizer and learning rate, or the number of epochs. How high performance can you achieve on the test split?

## Data

Start by downloading the datasets `toy_data_train.csv` and `toy_data_test.csv`.

Alternatively, if you have `histocc` installed, you can run the code below to prepare the data:

In [1]:
from histocc import DATASETS
from sklearn.model_selection import train_test_split


def download_and_prepare_data():
    keys = DATASETS['keys']()
    mapping = dict(keys[['hisco', 'code']].values)

    toydata = DATASETS['toydata']()
    toydata['label'] = toydata['hisco_1'].transform(lambda x: mapping[x])

    train, test = train_test_split(
        toydata[['occ1', 'label']],
        test_size=0.1,
        random_state=42,
        )

    train.to_csv('./toy_data_train.csv', index=False)
    test.to_csv('./toy_data_test.csv', index=False)


download_and_prepare_data()   

## Setup

#### Dependencies

In [2]:
from functools import partial

import torch

from torch import Tensor, nn
from torch.utils.data import Dataset, DataLoader

import pandas as pd

#### `Dataset` class

In [3]:
# These are all the individual tokens present in the toy dataset
CHARS_IN_TOYDATA = [' ', '"', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '¢', '£', '©', '¬', 'Â', 'Ã', 'â', 'œ', 'š', 'ž', '‚', '„', '€']
MAP_CHAR_IDX = {char: idx for idx, char in enumerate(CHARS_IN_TOYDATA, start=2)}

def tokenize(hisco: str, max_len: int) -> list[int]:
    encoded = [MAP_CHAR_IDX.get(char, 0) for char in hisco]
    encoded = encoded[:max_len]
    encoded += [1] * (max_len - len(encoded))

    return encoded


class HISCODataset(Dataset):
    def __init__(self, dataset: pd.DataFrame):
        super().__init__()

        self.dataset = dataset
        self.tokenizer = partial(tokenize, max_len=32)

    def __len__(self) -> int:
        return len(self.dataset)

    def __getitem__(self, item: int) -> dict[str, str | Tensor]:
        record = self.dataset.iloc[item]
        encoded = self.tokenizer(record.occ1)

        package = {
            'occ1': record.occ1,
            'encoded': torch.tensor(encoded, dtype=torch.long),
            'label': torch.tensor(record.label, dtype=torch.long),
        }

        return package

#### Load and prepare data

In [4]:
import os

from dirs import DATA_DIR

train_data = pd.read_csv(os.path.join(DATA_DIR, 'toy_data_train.csv'))
test_data = pd.read_csv(os.path.join(DATA_DIR, 'toy_data_test.csv'))

train_dataset = HISCODataset(train_data)
test_dataset = HISCODataset(test_data)

train_data_loader = DataLoader(train_dataset, batch_size=32)
test_data_loader = DataLoader(test_dataset, batch_size=32)

In [5]:
# (Optional) load in label <-> HISCO code mapping. Requires `histocc`
from histocc import DATASETS

keys = DATASETS['keys']()
map_hisco_label = dict(keys[['hisco', 'code']].values)
map_label_hisco = {v: k for k, v in map_hisco_label.items()}

## Exercise 1

In [6]:
class HISCOClassifierGRU(nn.Module):
    def __init__(self, vocab_size: int = 100, hidden_size: int = 128):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.classifier = nn.Linear(hidden_size, 1919)

    def forward(self, input_seq: Tensor) -> Tensor:
        out = self.embedding(input_seq)
        out, _ = self.gru(out)
        out = out[:, -1, :]
        out = self.classifier(out)

        return out


In [7]:
model_baseline = HISCOClassifierGRU()
optimizer_baseline = torch.optim.AdamW(model_baseline.parameters(), lr=0.01)

loss_fn = torch.nn.CrossEntropyLoss()

In [8]:
def train_epoch(model, optimizer, data_loader, loss_fn):
    model.train()

    for batch in data_loader:
        optimizer.zero_grad()
        
        out = model(batch['encoded']) # make predictions
        loss = loss_fn(out, batch['label']) # calculate loss
        loss.backward() # calculate derivatives

        optimizer.step() # update network parameters

In [9]:
@torch.no_grad
def evaluate(model, data_loader):
    model.eval()

    total_correct = 0 # keep count of correct predictions
    total_count = 0 # keep count of total number of predictions

    for batch in data_loader:
        out = model(batch['encoded']).argmax(1)

        total_correct += (out == batch['label']).sum().item()
        total_count += batch['label'].size(0)

    return total_correct / total_count # calculate accuracy

In [10]:
for epoch in range(1, 11):
    train_epoch(model_baseline, optimizer_baseline, train_data_loader, loss_fn)
    acc = evaluate(model_baseline, test_data_loader)

    print(f'Trained for {epoch} epochs. Validation accuracy: {100 * acc}%')

Trained for 1 epochs. Validation accuracy: 64.0%
Trained for 2 epochs. Validation accuracy: 70.8%
Trained for 3 epochs. Validation accuracy: 73.2%
Trained for 4 epochs. Validation accuracy: 72.3%
Trained for 5 epochs. Validation accuracy: 72.6%
Trained for 6 epochs. Validation accuracy: 74.1%
Trained for 7 epochs. Validation accuracy: 75.1%
Trained for 8 epochs. Validation accuracy: 76.4%
Trained for 9 epochs. Validation accuracy: 76.8%
Trained for 10 epochs. Validation accuracy: 76.6%


## Exercise 2

In [11]:
class HISCOClassifierSimpleRNN(nn.Module):
    def __init__(self, vocab_size: int = 100, hidden_size: int = 128):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.classifier = nn.Linear(hidden_size, 1919)

    def forward(self, input_seq: Tensor) -> Tensor:
        out = self.embedding(input_seq)
        out, _ = self.rnn(out)
        out = out[:, -1, :]
        out = self.classifier(out)

        return out


class HISCOClassifierLSTM(nn.Module):
    def __init__(self, vocab_size: int = 100, hidden_size: int = 128):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.classifier = nn.Linear(hidden_size, 1919)

    def forward(self, input_seq: Tensor) -> Tensor:
        out = self.embedding(input_seq)
        out, _ = self.lstm(out)
        out = out[:, -1, :]
        out = self.classifier(out)

        return out

In [12]:
model_simple = HISCOClassifierSimpleRNN()
optimizer_simple = torch.optim.AdamW(model_simple.parameters(), lr=0.01)

In [13]:
model_lstm = HISCOClassifierLSTM()
optimizer_lstm = torch.optim.AdamW(model_lstm.parameters(), lr=0.01)

In [14]:
for epoch in range(1, 11):
    train_epoch(model_simple, optimizer_simple, train_data_loader, loss_fn)
    acc = evaluate(model_simple, test_data_loader)

    print(f'Trained "SimpleRNN" model for {epoch} epochs. Validation accuracy: {100 * acc}%')

Trained "SimpleRNN" model for 1 epochs. Validation accuracy: 9.4%
Trained "SimpleRNN" model for 2 epochs. Validation accuracy: 10.100000000000001%
Trained "SimpleRNN" model for 3 epochs. Validation accuracy: 10.299999999999999%
Trained "SimpleRNN" model for 4 epochs. Validation accuracy: 10.299999999999999%
Trained "SimpleRNN" model for 5 epochs. Validation accuracy: 10.4%
Trained "SimpleRNN" model for 6 epochs. Validation accuracy: 9.2%
Trained "SimpleRNN" model for 7 epochs. Validation accuracy: 9.3%
Trained "SimpleRNN" model for 8 epochs. Validation accuracy: 9.4%
Trained "SimpleRNN" model for 9 epochs. Validation accuracy: 9.3%
Trained "SimpleRNN" model for 10 epochs. Validation accuracy: 9.3%


In [15]:
for epoch in range(1, 11):
    train_epoch(model_lstm, optimizer_lstm, train_data_loader, loss_fn)
    acc = evaluate(model_lstm, test_data_loader)

    print(f'Trained "LSTM" model for {epoch} epochs. Validation accuracy: {100 * acc}%')

Trained "LSTM" model for 1 epochs. Validation accuracy: 9.2%
Trained "LSTM" model for 2 epochs. Validation accuracy: 14.000000000000002%
Trained "LSTM" model for 3 epochs. Validation accuracy: 19.5%
Trained "LSTM" model for 4 epochs. Validation accuracy: 25.6%
Trained "LSTM" model for 5 epochs. Validation accuracy: 35.8%
Trained "LSTM" model for 6 epochs. Validation accuracy: 40.8%
Trained "LSTM" model for 7 epochs. Validation accuracy: 44.5%
Trained "LSTM" model for 8 epochs. Validation accuracy: 45.6%
Trained "LSTM" model for 9 epochs. Validation accuracy: 46.2%
Trained "LSTM" model for 10 epochs. Validation accuracy: 47.4%


## Exercise 3

In [16]:
model_lstm_new_optimizer = HISCOClassifierLSTM()
optimizer_lstm_new_optimizer = torch.optim.RMSprop(model_lstm_new_optimizer.parameters(), lr=0.01)

In [17]:
for epoch in range(1, 11):
    train_epoch(model_lstm_new_optimizer, optimizer_lstm_new_optimizer, train_data_loader, loss_fn)
    acc = evaluate(model_lstm_new_optimizer, test_data_loader)

    print(f'Trained "LSTM" (RMSprop) model for {epoch} epochs. Validation accuracy: {100 * acc}%')

Trained "LSTM" (RMSprop) model for 1 epochs. Validation accuracy: 18.7%
Trained "LSTM" (RMSprop) model for 2 epochs. Validation accuracy: 27.500000000000004%
Trained "LSTM" (RMSprop) model for 3 epochs. Validation accuracy: 41.9%
Trained "LSTM" (RMSprop) model for 4 epochs. Validation accuracy: 46.7%
Trained "LSTM" (RMSprop) model for 5 epochs. Validation accuracy: 51.7%
Trained "LSTM" (RMSprop) model for 6 epochs. Validation accuracy: 55.900000000000006%
Trained "LSTM" (RMSprop) model for 7 epochs. Validation accuracy: 58.099999999999994%
Trained "LSTM" (RMSprop) model for 8 epochs. Validation accuracy: 59.8%
Trained "LSTM" (RMSprop) model for 9 epochs. Validation accuracy: 61.8%
Trained "LSTM" (RMSprop) model for 10 epochs. Validation accuracy: 63.2%


## Exercise 4

In [18]:
class FancyHISCOClassifier(nn.Module):
    def __init__(self, vocab_size: int = 100, hidden_size: int = 128, dropout: float = 0.0):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.lstm = nn.GRU(
            hidden_size, 
            hidden_size, 
            batch_first=True,
            num_layers=2,
            )
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(hidden_size, 1919)

    def forward(self, input_seq: Tensor) -> Tensor:
        out = self.embedding(input_seq)
        out, _ = self.lstm(out)
        out = out[:, -1, :]
        out = self.dropout(out)
        out = self.classifier(out)

        return out

In [19]:
device = 'cuda'

model_fancy = FancyHISCOClassifier(hidden_size=128, dropout=0.5).to(device)
optimizer_fancy = torch.optim.AdamW(model_fancy.parameters(), lr=0.01)
scheduler_fancy = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_fancy, T_max=50)

In [20]:
def train_epoch_select_device(model, optimizer, data_loader, loss_fn, device):
    model.train()

    for batch in data_loader:
        optimizer.zero_grad()
        
        out = model(batch['encoded'].to(device)) # make predictions
        loss = loss_fn(out, batch['label'].to(device)) # calculate loss
        loss.backward() # calculate derivatives

        optimizer.step() # update network parameters

In [21]:
@torch.no_grad
def evaluate_select_device(model, data_loader, device):
    model.eval()

    total_correct = 0 # keep count of correct predictions
    total_count = 0 # keep count of total number of predictions

    for batch in data_loader:
        out = model(batch['encoded'].to(device)).argmax(1).cpu()

        total_correct += (out == batch['label']).sum().item()
        total_count += batch['label'].size(0)

    return total_correct / total_count # calculate accuracy

In [22]:
for epoch in range(1, 51):
    train_epoch_select_device(model_fancy, optimizer_fancy, train_data_loader, loss_fn, device)
    acc = evaluate_select_device(model_fancy, test_data_loader, device)

    print(f'Trained "fancy" model for {epoch} epochs. Validation accuracy: {100 * acc}%')

    scheduler_fancy.step()

Trained "fancy" model for 1 epochs. Validation accuracy: 57.4%
Trained "fancy" model for 2 epochs. Validation accuracy: 61.8%
Trained "fancy" model for 3 epochs. Validation accuracy: 66.9%
Trained "fancy" model for 4 epochs. Validation accuracy: 71.89999999999999%
Trained "fancy" model for 5 epochs. Validation accuracy: 72.2%
Trained "fancy" model for 6 epochs. Validation accuracy: 71.39999999999999%
Trained "fancy" model for 7 epochs. Validation accuracy: 73.0%
Trained "fancy" model for 8 epochs. Validation accuracy: 73.6%
Trained "fancy" model for 9 epochs. Validation accuracy: 74.2%
Trained "fancy" model for 10 epochs. Validation accuracy: 75.2%
Trained "fancy" model for 11 epochs. Validation accuracy: 76.0%
Trained "fancy" model for 12 epochs. Validation accuracy: 78.10000000000001%
Trained "fancy" model for 13 epochs. Validation accuracy: 75.8%
Trained "fancy" model for 14 epochs. Validation accuracy: 76.0%
Trained "fancy" model for 15 epochs. Validation accuracy: 77.2%
Trained "f